In [1]:
import pysindy as ps

import deepSI
from deepSI.fit_systems import SS_encoder_general
from deepSI.fit_systems.encoders import default_encoder_net, default_state_net, default_output_net

import torch
from torch import nn

import numpy as np

from sklearn.preprocessing import PolynomialFeatures

from scipy.io import loadmat
import os

import deepSI
from deepSI import System_data

from utils import load_data

c:\Users\20173928\Miniconda3\envs\sindy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
save_dir = r"C:\Users\20173928\OneDrive - TU Eindhoven\Documents\Master\thesis\mscth\data\own_data"

out = loadmat(os.path.join(save_dir,'MSD_x_data.mat'))
x_data = out['x']

out = loadmat(os.path.join(save_dir,'MSD_u_data.mat'))
u_data = out['u']

out = loadmat(os.path.join(save_dir,'MSD_y_data.mat'))
y_data = out['y']

train, test = System_data(u=u_data[:-1000,0],y=y_data[:-1000,0]), System_data(u=u_data[-1000:,0],y=y_data[-1000:,0])

In [3]:
x_data, u_data, y_data, th_data = load_data(pc=0)

train, test = System_data(u=u_data[:10000,0],y=x_data[:10000,:]), System_data(u=u_data[-1000:],y=x_data[-1000:,:])

train.y.shape, train.u.shape, test.y.shape, test.u.shape

((10000, 2), (10000,), (1000, 2), (1000, 1))

In [4]:
class SS_encoder_general_eq(SS_encoder_general):
    def __init__(self, nx=10, na=20, nb=20, feedthrough=False, \
        e_net=default_encoder_net, f_net=default_state_net, h_net=default_output_net, \
        e_net_kwargs={},           f_net_kwargs={},         h_net_kwargs={}, na_right=0, nb_right=0, \
        gamma=1e-4):

        super(SS_encoder_general_eq, self).__init__()
        self.nx, self.na, self.nb = nx, na, nb
        self.k0 = max(self.na,self.nb)
        
        self.e_net = e_net
        self.e_net_kwargs = e_net_kwargs

        self.f_net = f_net
        self.f_net_kwargs = f_net_kwargs

        self.h_net = h_net
        self.h_net_kwargs = h_net_kwargs

        self.feedthrough = feedthrough
        self.na_right = na_right
        self.nb_right = nb_right
        ######################################
        # args added for feature transform and
        # regurlarization
        self.gamma = gamma
        ######################################

    def init_nets(self, nu, ny): # a bit weird
        na_right = self.na_right if hasattr(self,'na_right') else 0
        nb_right = self.nb_right if hasattr(self,'nb_right') else 0
        self.encoder = self.e_net(nb=(self.nb+nb_right), nu=nu, na=(self.na+na_right), ny=ny, nx=self.nx, **self.e_net_kwargs)
        ######################################
        ###### change fn intialization #######
        self.fn     =      self.f_net(nx=self.nx, nu=nu, **self.f_net_kwargs)
        ######################################
        if self.feedthrough:
            self.hn =      self.h_net(nx=self.nx, ny=ny, nu=nu,                     **self.h_net_kwargs) 
        else:
            self.hn =      self.h_net(nx=self.nx, ny=ny,                            **self.h_net_kwargs) 

    def loss(self, uhist, yhist, ufuture, yfuture, loss_nf_cutoff=None, **Loss_kwargs):
        x = self.encoder(uhist, yhist) #initialize Nbatch number of states
        # print(torch.max(x))
        # print()
        errors = []
        for y, u in zip(torch.transpose(yfuture,0,1), torch.transpose(ufuture,0,1)): #iterate over time
            error = nn.functional.mse_loss(y, self.hn(x,u) if self.feedthrough else self.hn(x))
            ##################################
            ## add penalty to weights in fn ##
            # params = [*self.fn.parameters()]
            # weights = [x.view(-1) for x in params][0]
            # error += self.gamma*torch.norm(weights, 1)
            ##################################
            errors.append(error) #calculate error after taking n-steps
            if loss_nf_cutoff is not None and error.item()>loss_nf_cutoff:
                print(len(errors), end=' ')
                break
            x = self.fn(x,u) #advance state. 
            
        return torch.mean(torch.stack(errors))

In [5]:
class identity(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

    def forward(self, *input):
        return input
    
class simple_Linear(torch.nn.Module):
    def __init__(self, nx, nu, **kwargs):
        super(simple_Linear, self).__init__()

        self.nx = nx
        self.nu = kwargs['u']

        self.feature_library = kwargs['feature_library']
        test_sample = torch.rand(1,self.nx+self.nu, requires_grad=True)
        self.nf = (self.feature_library.fit_transform(test_sample)).shape[1]
        
        self.layer = nn.Linear(self.nf, nx, bias=False)
        

    def forward(self, x, u):
        x = torch.hstack((x, u.unsqueeze(1)))
        Theta = self.feature_library.fit_transform(x)
        out = self.layer(Theta)
        return out

In [6]:
class feature_library():
    def __init__(
            self,
            functions,
            interaction_only=True
    ):
        self.functions = functions
        self.interaction_only = interaction_only

    def fit_transform(self, X):
        # off set
        out_feature = ((X[:,0])**0).unsqueeze(1)
        if self.interaction_only:
            for f in self.functions:
                out_feature = torch.hstack((out_feature, f(X)))
            return out_feature
        

In [7]:
def f(x):
  return x

def f2(x):
  return x**2

def sin(x):
  return np.sin(x)

def f3(x):
  return x**3

In [8]:
# functions = [lambda x:x,
#              lambda x:x**2,
#              lambda x:x**3]
functions = [f, f3]

poly = feature_library(functions=functions)

In [12]:
# fit_sys = deepSI.fit_systems.SS_encoder_general(nx=2, na=50, nb=50)

nx, nu = 2, 1 # state dimension and inputs
na, nb = 0, 0

f_net = simple_Linear
f_net_kwargs= f_net_kwargs={"feature_library": poly, "u": nu, "nf": 10}

h_net = identity
h_net_kwargs = {}

fit_sys = SS_encoder_general_eq(nx=2, na=na, nb=nb, \
                                f_net=f_net, f_net_kwargs=f_net_kwargs,\
                                e_net=identity,\
                                h_net=identity)


fit_sys.fit(train, test, epochs=1, batch_size = 2, optimizer_kwargs={"lr": 1e-2}, loss_kwargs=dict(nf=100))

Initilizing the model and optimizer
Size of the training array =  22.7 MB
N_training_samples = 9901, batch_size = 2, N_batch_updates_per_epoch = 4950


TypeError: identity.forward() takes 2 positional arguments but 3 were given